## Word Embedding
---


In [141]:
# Import
import pandas as pd
import PyPDF2

pdfFileObj = open('20221118-FULL.pdf', 'rb') #Create pdf object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

In [142]:
allText = ""
for i in range(100): #Iterating over n pages 
    pageObj = pdfReader.getPage(i)
    page = pageObj.extractText()
    allText += page #Store page content into string variable

allText;

#### Split Text into persons

---

In [143]:
allText = allText.replace(';','')
allText[:1000]

'EUROPEAN COMMISSION\nFinancial Stability, Financial Services and Capital Markets Union (DG FISMA)\nEuropean Union\nConsolidated Financial Sanctions List\nThis list has been updated on 18/11/2022 11:37European Union Consolidated Financial Sanctions List\nTable of contents\n1. Introduction3 \n2. Individuals or persons3 \n3. Entities or groups631 \n4. Disclaimer792 \nPage 2 on 792European Union Consolidated Financial Sanctions List\n1. INTRODUCTION\nThe present document contains the Consolidated List of persons, groups and entities subject, under EU Sanctions, to an asset freeze and the prohibition to\nmake funds and economic resources available to them. The latest version of this file is here.\n2. INDIVIDUALS OR PERSONS\nEU reference number: EU.1787.1\nLegal basis: 2017/404 (OJ L63)\nProgramme: AFG - Afghanistan\nIdentity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Name/A

In [144]:
alltext_string_all = allText.replace('Legal basis',';Legal basis').replace('Identity information',';Identity information').replace('Programme', ';Programme').replace('Birth information',';Birth information').replace('Citizenship information',';Citizenship information').replace('Contact information',';Contact information').replace('Remark',';Remark')
alltext_string_all[:1000]


'EUROPEAN COMMISSION\nFinancial Stability, Financial Services and Capital Markets Union (DG FISMA)\nEuropean Union\nConsolidated Financial Sanctions List\nThis list has been updated on 18/11/2022 11:37European Union Consolidated Financial Sanctions List\nTable of contents\n1. Introduction3 \n2. Individuals or persons3 \n3. Entities or groups631 \n4. Disclaimer792 \nPage 2 on 792European Union Consolidated Financial Sanctions List\n1. INTRODUCTION\nThe present document contains the Consolidated List of persons, groups and entities subject, under EU Sanctions, to an asset freeze and the prohibition to\nmake funds and economic resources available to them. The latest version of this file is here.\n2. INDIVIDUALS OR PERSONS\nEU reference number: EU.1787.1\n;Legal basis: 2017/404 (OJ L63)\n;Programme: AFG - Afghanistan\n;Identity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Nam

In [145]:
testListSplit = alltext_string_all.split('reference number: EU')
testListSplit[:10]

for i in testListSplit: #Visualization for information
    print(i)
    print('--------------------------------')



EUROPEAN COMMISSION
Financial Stability, Financial Services and Capital Markets Union (DG FISMA)
European Union
Consolidated Financial Sanctions List
This list has been updated on 18/11/2022 11:37European Union Consolidated Financial Sanctions List
Table of contents
1. Introduction3 
2. Individuals or persons3 
3. Entities or groups631 
4. Disclaimer792 
Page 2 on 792European Union Consolidated Financial Sanctions List
1. INTRODUCTION
The present document contains the Consolidated List of persons, groups and entities subject, under EU Sanctions, to an asset freeze and the prohibition to
make funds and economic resources available to them. The latest version of this file is here.
2. INDIVIDUALS OR PERSONS
EU 
--------------------------------
.1787.1
;Legal basis: 2017/404 (OJ L63)
;Programme: AFG - Afghanistan
;Identity information:
• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  
• Name/Alias: Khalifa  
• Name/Alias: Saraj Haqani  
• Name/Alias: 

In [146]:
# Create dataframe
dbn_df = pd.DataFrame([sub.split(";") for sub in testListSplit])
dbn_df.columns = ['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information', 'Remark','','','','','','','','']
dbn_df.fillna("",inplace=True)



In [147]:
dbn_df['Identity information'][1]

'Identity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Name/Alias: Siraj Haqani  \n• Name/Alias: Serajuddin Haqani  \n• Name/Alias: Siraj Haqqani  \n'

## Dataframe Clenup 
---

In [148]:
def cleanupDataframe(dataframe):
    dataframe['Legal basis'] = dataframe['Legal basis'].apply(lambda x: x.replace('Legal basis: ','').replace('\n',''))
    dataframe['Programme'] = dataframe['Programme'].apply(lambda x: x.replace('Programme: ','').replace('\n',''))
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.replace('Identity information:','').replace('\n','').replace('Name/Alias:',';').replace('•',''))
    
    
    return dataframe


In [149]:
# teststring = 'das; ist; ein ; string;'
# [x for x in teststring.split(';')]

dbn_df['Identity information'] = dbn_df['Identity information'].apply(lambda content: [x for x in content.split(';')])

In [150]:
dbn_df

,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information,Remark,,,,,,,,
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,[],,,,,,,,,,,,
1,.1787.1\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,[Identity information:\n• Name/Alias: Sirajudd...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi...",Remark: Heading the Haqqani Network as of late...,,,,,,,,
2,.2272.60\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,[Identity information:\n• Name/Alias: Nasirudd...,Birth information:\n• Birth date: Circa from 1...,"Remark: Neka District, Paktika Province, Afgha...",Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan \n,Remark: A leader of the Haqqani Network (TAe.0...,,,,,,,
3,.2274.87\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,[Identity information:\n• Name/Alias: Gul Agha...,Birth information:\n• Birth date: Circa 1972 ...,Contact information: \n• Address: Pakistan \n...,Remark: Belongs to Ishaqzai tribe. INTERPOL-UN...,,,,,,,,,
4,.2291.0\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,[Identity information:\n• Name/Alias: Amir Abd...,Birth information:\n• Birth date: Circa 1972 ...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka...",Remark: Believed to be in Afghanistan/Pakistan...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,.3631.49\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,[Identity information:\n• Name/Alias: Mathieu ...,Birth information:\n• Birth date: 08/10/1970 ...,"Remark: Place of birth: Bunia, Ituri\nProvince...",Citizenship information:\n• Citizenship: Congo...,"Contact information: \n• Address: Congo, Democ...",Remark: Mathieu Chui Ngudjolo was the Chief of...,,,,,,,
384,.3632.14\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo...,[Identity information:\n• Name/Alias: Floriber...,Birth information:\n• Birth date: 23/05/1971 ...,Citizenship information:\n• Citizenship: Congo...,Remark: President of FNI\nEU,,,,,,,,,
385,.3633.76\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,[Identity information:\n• Name/Alias: 'Chairma...,Birth information:\n• Birth date: 06/02/1967 ...,Citizenship information:\n• Citizenship: Congo...,,,,,,,,,,
386,.3634.41\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,[Identity information:\n• Name/Alias: Fred Ira...,"Birth information:\n• Birth place: Rwanda, Kin...",Citizenship information:\n• Citizenship: Rwanda\n,"Contact information: \n• Address: Congo, Democ...",Remark: As of June 2016\nEU,,,,,,,,


## Word2Vec
---

In [151]:
import string
import nltk
from nltk.corpus import brown
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
 
#Preprocessing


 

In [157]:
dbn_df['Identity information'][1]

['Identity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Name/Alias: Siraj Haqani  \n• Name/Alias: Serajuddin Haqani  \n• Name/Alias: Siraj Haqqani  \n']